# Toronto Territory Tussle
This notebook will be used for the capstone project in the IBM Applied Data Science Specialization on Coursera.

## 1. Download and Explore Dataset (Week 3)

First of all we import the _pandas_ library to load and manipulate datasets.

In [1]:
import pandas as pd # library for data analsysis

print('Pandas imported.')

Pandas imported.


In the next step, we load the wikitable from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M into a _pandas_ dataframe.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


---
Rename the first column to adhere to the specifications giving in the excercise:

In [3]:
df.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


---
Dropping all rows that do not have an assigned borough:

In [4]:
df = df[df["Borough"]!="Not assigned"].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


---
Checking duplicate postal codes:

In [5]:
row_no = df.shape[0]
pc_no = df["PostalCode"].unique().shape[0]

print("Number of rows in dataframe:", row_no)
print("Number of unique postal codes:", pc_no)

print("There are", row_no - pc_no, "duplicate postal codes.")

Number of rows in dataframe: 103
Number of unique postal codes: 103
There are 0 duplicate postal codes.


It appears that the Wikipedia page has been updated since the creation of this week's excercise

---
Now we check whether there are any remaining rows in the dataframe with not assigned neighbourhoods.

In [6]:
nA_neighbourhoods = df[df["Neighbourhood"]=="Not assigned"].shape[0]
print("There are", nA_neighbourhoods ,"entries with not assigned neighbourhoods")

There are 0 entries with not assigned neighbourhoods


There is no need to change any entries in the dataframe.

---
According to the final excercise of this week, we print the number of rows in the dataframe:

In [7]:
row_no = df.shape[0]
print("There are", row_no, "rows in the dataframe.")

There are 103 rows in the dataframe.
